In [1]:
import os 
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import generate_random_spectrum as grs 

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

In [2]:
print(f' {datetime.datetime.today():%H:%M:%S - %m/%d/%Y}')

 20:00:19 - 05/28/2024


### Шаблон таблиц

In [3]:
df_peaks = pd.DataFrame(
    columns=['id', 'height', 'area', 'fwhm', 'position', 'intensity', 'asym', 'gaussshare', 'noise_coef']
)
df_curves = pd.DataFrame(
    columns=['id_peak', 'curve_type', *np.arange(1025)]
)

### Параметры кривых

In [4]:
max_amplitudes = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
noise_coefs = [0, 0.5, 1, 2, 4, 8]
line_types = ('peak', 'noise', 'bl', 'fd', 'sd')
current_id = 0
current_row_curves = 0
np.random.seed(4)

log_file = open('logs.txt', 'a')

###  Запуск расчета данных

In [ ]:
for trial in range(1000):
    for max_amplitude in max_amplitudes:
        for noise_coef in noise_coefs:
            current_multipeak, current_noise, current_baseline, first_der, second_der = grs.generate_the_spectrum(
                number_of_peaks=1, function='asym_pV',
                return_order=3, 
                max_amplitude=max_amplitude,
                noise_coef=noise_coef, 
                baseline_multiplier=1e-5, intensity=0.1,
                display=0
            )
            peak = (
                current_id, current_multipeak.peaks[0].peak_height, 
                current_multipeak.peaks[0].peak_area, 
                *current_multipeak.params.loc[['fwhm', 'position', 'intensity', 'asym', 'gaussshare'], 0].values, 
                noise_coef
            )
            df_peaks.loc[current_id] = peak
            
            lines = (current_multipeak.peaks[0].curve, current_noise, current_baseline, first_der, second_der)
            for name, line_data in zip(line_types, lines):
                curve = (current_id, name, *line_data)
                df_curves.loc[current_row_curves] = curve
                current_row_curves += 1
            
            if current_id % 500 == 0:
                print(f"|{datetime.datetime.today():%H:%M:%S - %m/%d/%Y}|{trial:>6}|{max_amplitude:>6}|{noise_coef:>6}|", 
                      file=log_file, flush=True
                     )
                df_peaks.to_csv('./data/df_peaks.csv')
                df_curves.to_csv('./data/df_curves.csv')
            current_id += 1
            